In [2]:
import time
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier

from tianchi.user_location_prediction.pysrc.feature import *
from tianchi.user_location_prediction.pysrc.load import *

In [3]:
sample_i = load_sample_test()
sample_wifi_i = load_sample_wifi_test()
shop_i = load_shop_test()

In [4]:
sample_wifi_i.head(3)

,sample_id,wifi_id,signal_power,signal_flag
0,0,b_6396480,0.510714,0.1
1,0,b_41124514,0.341071,0.1
2,0,b_28723327,0.305357,0.1


In [5]:
sample_i.head()

,sample_id,row_id,mall_id,user_id,lgt,ltt,time,wifi_infos,shop_id
0,0,NaN,m_1409,u_376,122.308291,32.088040,2017-08-06 21:20:00,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,1,NaN,m_1409,u_376,122.308162,32.087970,2017-08-06 21:20:00,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,68,NaN,m_1409,u_13782,122.309295,32.088638,2017-08-14 20:40:00,b_11592648|-74|false;b_56326068|-61|false;b_29...,s_52283
3,80,NaN,m_1409,u_18171,122.310365,32.087681,2017-08-11 15:30:00,b_41870824|-84|false;b_51825273|-76|false;b_41...,s_288430
4,156,NaN,m_1409,u_29343,122.310139,32.087676,2017-08-14 13:30:00,b_55113661|-76|false;b_52642831|-73|false;b_38...,s_288430


In [ ]:
def construct_batch(sample_df, sample_wifi_df, shop_df):
    feat_list = []

    weekday_feat = weekday_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(weekday_feat)

    hour_feat = hour_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(hour_feat)

    spaceloc_feat = spaceloc_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_feat)

    spaceloc_dist_feat = spaceloc_dist_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(spaceloc_dist_feat)

    wifi_power_feat = wifi_power_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_power_feat)

    wifi_powerloc_dist_feat = wifi_powerloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_power_feat)
    feat_list.append(wifi_powerloc_dist_feat)

    wifi_flag_feat = wifi_flag_featext(sample_df, sample_wifi_df, shop_df)
    feat_list.append(wifi_flag_feat)

    wifi_flagloc_dist_feat = wifi_flagloc_dist_featext(sample_df, sample_wifi_df, shop_df, wifi_flag_feat)
    feat_list.append(wifi_flagloc_dist_feat)

    feat = np.concatenate(feat_list, axis=1)
    train_idx = sample_df[sample_df.row_id.isnull()].index.tolist()[-1] + 1

    train_x = feat[:train_idx]
    train_y = sample_df[sample_df.row_id.isnull()]['shop_id'].values
    test_x = feat[train_idx:]
    test_row_id = list(sample_df[-sample_df.row_id.isnull()]['row_id'].values)

    return train_x, train_y, test_x, test_row_id